In [1]:
import os
import random

import torch
from PIL import Image
from diffusers import DPMSolverMultistepScheduler
from diffusers.utils import make_image_grid

from models.combined_pipeline import CombinedPipeline


In [2]:
pipe = CombinedPipeline.from_pretrained('../models/weights/img2img-painting', torch_dtype=torch.float16, variant='fp16', use_safetensors=True)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
lora_model_path = '../models/weights/img2img-painting-lora/pytorch_lora_weights.safetensors'
# lora_model_path = '../models/weights/_/models/weights/img2img-painting-lora/checkpoint-500/pytorch_model.bin'
# pipe.unet.load_attn_procs(lora_model_path)
pipe.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

In [3]:
def save_images(images, path=''):
    for i, image in enumerate(images):
        image.save(path + f'image{i}.png')

def create_img(source_image, target_image):
    text = ['shirt', 'pants', 'shorts', 'shoes', 't-shirt',
            'sweatshirt', "cowboy boot", "cowboy hat",
            'crash helmet', 'cardigan', 'blouse', 'jacket', 'jeans',]
    condition = ['head, neck', 'hair', 'face', 'hand',]
    return pipe(source_image, target_image,
                source_prompt=text,
                source_negative_prompt=condition,
                target_prompt=text,
                target_negative_prompt=condition,
                guidance_scale=9, num_inference_steps=100,
                mask_threshold=0.4,
                cross_attention_kwargs={"scale": 0.5},)

In [4]:
input_folder = '../cache/in/'
output_folder = '../cache/out/'

In [5]:
from datasets import load_from_disk

DATASET_DIR = 'SaffalPoosh/deepFashion-with-masks'
dataset = load_from_disk('../data/dataset/train/')
start = 119
count = 5
save_images(dataset['train'][start: start + count]['images'], path=input_folder)

In [6]:
paths = os.listdir(input_folder)
random.shuffle(paths)

images = [Image.open(input_folder + path) for path in paths]
generated_images, couple = [], []

for i in range(len(images)):
    for j in range(i + 1, len(images)):
        images[i], images[j] = images[i].resize((512, 512)), images[j].resize((512, 512))
        generated_images.extend([
            make_image_grid([*create_img(images[i], images[j]),
                             images[i], images[j]], 3, 2),
            make_image_grid([*create_img(images[j], images[i]),
                             images[j], images[i]], 3, 2)
        ])

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
save_images(generated_images, path=output_folder)